In [1]:
# Import Dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Connecting to S3 bucket
import s3fs

# Create pandas dataframe from the csv file
df = pd.read_csv('https://jamesliu-databootcamp-bucket.s3.us-east-2.amazonaws.com/pseof_all.csv')

In [9]:
df.head()

,agg_level_pseo,inst_level,institution,degree_level,cip_level,cipcode,grad_cohort,grad_cohort_years,geo_level,geography,...,y10_grads_nme,status_y1_grads_emp,status_y1_grads_emp_instate,status_y5_grads_emp,status_y5_grads_emp_instate,status_y10_grads_emp,status_y10_grads_emp_instate,status_y1_grads_nme,status_y5_grads_nme,status_y10_grads_nme
0,38,I,00105100,5,A,0,0,3,N,0,...,7281.0,1,1,1,1,1,1,1,1,1
1,38,I,00105100,7,A,0,0,5,N,0,...,1149.0,1,1,1,1,1,1,1,1,1
2,38,I,00105100,17,A,0,0,5,N,0,...,191.0,1,1,1,1,1,1,1,1,1
3,38,I,00105100,18,A,0,0,5,N,0,...,256.0,1,1,1,1,1,1,1,1,1
4,38,I,00105200,2,A,0,0,5,N,0,...,NaN,1,1,1,1,-1,-1,1,1,-1
